In [2]:
print("all ok")

all ok


In [3]:
from langchain_groq import ChatGroq

In [4]:
from dotenv import load_dotenv

In [41]:
load_dotenv()

True

In [6]:
model = ChatGroq( model="qwen/qwen3-32b")

In [7]:
print(model.invoke("What is the greatest truth no one knows?").content)

<think>
Okay, the user is asking about the greatest truth no one knows. Hmm, that's pretty abstract. Let me start by unpacking the question. They want a truth that's both significant and unknown to everyone. First, I need to consider different domains where such truths might exist—science, philosophy, personal experiences, etc. 

In science, there are many unanswered questions. For example, the nature of dark matter or the ultimate fate of the universe. But are these truly unknown to everyone? Scientists are working on them, so maybe not the "no one knows" part. Then there's the philosophical angle. What about existential truths? Like the meaning of life or consciousness? But those are more subjective. 

Maybe the user is looking for something more profound. The idea that there are truths we can't even conceive yet. Like the limits of human knowledge. Maybe something like Gödel's incompleteness theorems, suggesting there are mathematical truths we can't prove within the system. But aga

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [9]:
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [10]:
embedding_model.embed_query("what is the capital of Canada?")

[-0.01800224371254444,
 -0.010611603036522865,
 0.033239591866731644,
 -0.06110759451985359,
 -0.016292888671159744,
 0.004924595355987549,
 0.013344960287213326,
 -0.0021071049850434065,
 -0.0023149140179157257,
 -0.014550971798598766,
 -0.011767839081585407,
 -0.01574038341641426,
 -0.02077910490334034,
 -0.0014122184365987778,
 0.12613780796527863,
 -0.006469818763434887,
 -0.008973916992545128,
 0.0011495023500174284,
 0.02190861850976944,
 -0.02182980254292488,
 -0.0038364536594599485,
 0.013714619912207127,
 0.015500912442803383,
 -0.009586462751030922,
 -0.0029528706800192595,
 0.009915257804095745,
 0.0024443448055535555,
 -0.006325980182737112,
 0.009564211592078209,
 2.467856029397808e-05,
 -0.012669120915234089,
 0.005265390034765005,
 0.019763395190238953,
 -0.016844313591718674,
 -0.0033547631464898586,
 0.00010216909868177027,
 -0.0031274184584617615,
 0.004143101628869772,
 -0.014018218964338303,
 0.015676628798246384,
 -0.008014081977307796,
 -0.019438495859503746,
 0.0

In [11]:
len(embedding_model.embed_query("what is the capital of Canada?"))

3072

# Implementing RAG

## Data Ingestion

In [12]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
import os

In [14]:
file_path = os.path.join(os.getcwd(),"data","sample.pdf")

In [15]:
data = PyMuPDFLoader(file_path)

In [16]:
document = data.load()

In [17]:
len(document)

266

In [18]:
document

[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2013-07-01T15:30:52-05:00', 'source': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'file_path': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'total_pages': 266, 'format': 'PDF 1.6', 'title': 'The Total Money Makeover', 'author': 'Dave Ramsey', 'subject': '', 'keywords': '', 'moddate': '2013-07-15T14:28:01+05:30', 'trapped': '', 'modDate': "D:20130715142801+05'30'", 'creationDate': "D:20130701153052-05'00'", 'page': 0}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2013-07-01T15:30:52-05:00', 'source': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'file_path': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\

In [30]:
#this is experimental value. Kind of like hyperparameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len
)

In [31]:
docs = text_splitter.split_documents(document)

In [32]:
len(docs)

696

In [33]:
docs[19].metadata

{'producer': 'Adobe PDF Library 10.0.1',
 'creator': 'Adobe InDesign CS6 (Macintosh)',
 'creationdate': '2013-07-01T15:30:52-05:00',
 'source': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf',
 'file_path': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf',
 'total_pages': 266,
 'format': 'PDF 1.6',
 'title': 'The Total Money Makeover',
 'author': 'Dave Ramsey',
 'subject': '',
 'keywords': '',
 'moddate': '2013-07-15T14:28:01+05:30',
 'trapped': '',
 'modDate': "D:20130715142801+05'30'",
 'creationDate': "D:20130701153052-05'00'",
 'page': 13}

In [23]:
print(docs[19].page_content)

Introduction      xi
LIVE LIKE NO ONE ELSE.    IF YOU WILL LIVE LIKE NO ONE ELSE, LATER YOU CAN LIVE LIKE NO ONE ELSE.    IF YOU WILL LIVE LIKE NO ONE ELSE, LATER YOU CAN LIVE l
AN
has caused them to take action and claim ultimate victory over their finan­
cial struggles and worries—and to actually win! The number of people I 
meet across this nation who tell me this is the first book of any kind they 
have read in ten years is staggering. The Total Money Makeover is a book 
for everyone. It is for high-income earners, as I am. And it is for someone 
beginning at the bottom, which is where I once was.
You are about to read about a process, a proven plan, to win. You will


In [24]:
print(docs[20].page_content)

beginning at the bottom, which is where I once was.
You are about to read about a process, a proven plan, to win. You will 
find the plan to be very simple yet very inspiring. The principles are not 
mine. I stole them all from God and your grandmother. The principles are 
common sense, which isn’t so common anymore. The plan is mine—and 
I’m no genius—created by simply observing millions of lives with whom 
I have interacted through radio, TV, books, classes, the Internet, e-mail, 
podcasts, and our Live Events. I have successfully bottled common sense 
about money into a plan that anyone can do. And millions are!
When I first started talking about these principles years ago, I knew


In [25]:
from langchain.vectorstores import FAISS

In [34]:
len(embedding_model.embed_documents(docs[0].page_content))

63

In [35]:
vectorstore = FAISS.from_documents(docs, embedding_model)

GoogleGenerativeAIError: Error embedding content: 429 Resource has been exhausted (e.g. check quota).

## Using OpenAi embedding model

In [43]:
from langchain_openai import OpenAIEmbeddings

# Latest and most capable
embedding_model = OpenAIEmbeddings(
    #model="text-embedding-3-large",  # Most capable
    model="text-embedding-3-small",  # Faster and cheaper
    # model="text-embedding-ada-002",  # Legacy but still good
)

In [44]:
vectorstore = FAISS.from_documents(docs, embedding_model)

The below is a Retrieval process

In [ ]:
relevant_doc = vectorstore.similarity_search("what are the top 5 ways to save money?")

In [49]:
print(relevant_doc[0].page_content)

Save $1,000 Fast: Walk Before You Run      101
LIVE LIKE NO ONE ELSE.    IF YOU WILL LIVE LIKE NO ONE ELSE, LATER YOU CAN LIVE LIKE NO ONE ELSE.    IF YOU WILL LIVE LIKE NO ONE ELSE, LATER YOU CAN LIVE l
AN
Hide It
When you get the $1,000, hide it. You can’t keep the money handy because 
it will get spent. If your $1,000 from Baby Step One is in the underwear 
drawer, the pizza man will get it. No, the pizza man isn’t in your under­
wear drawer, but you will impulse-buy something if the money is easily 
accessible. You can put it in the bank savings account, but it cannot become 
overdraft protection. Don’t attach the savings account to your checking to 
protect you from overdrafting because then your emergency fund will get 
spent on impulse. I have had to learn to protect myself from me. We are not 
putting the money in the bank to earn money, but rather to make it hard to 
get. Since $1,000 at 4 percent earns only $40 per year, you aren’t getting


In [ ]:
relevant_doc = vectorstore.similarity_search("what are the top 5 ways to save money?", k=10)

In [51]:
relevant_doc

[Document(id='db675a61-ae07-4aef-b4a4-8ebd77bc50f8', metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2013-07-01T15:30:52-05:00', 'source': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'file_path': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'total_pages': 266, 'format': 'PDF 1.6', 'title': 'The Total Money Makeover', 'author': 'Dave Ramsey', 'subject': '', 'keywords': '', 'moddate': '2013-07-15T14:28:01+05:30', 'trapped': '', 'modDate': "D:20130715142801+05'30'", 'creationDate': "D:20130701153052-05'00'", 'page': 129}, page_content='Save $1,000 Fast: Walk Before You Run      101\nLIVE LIKE NO ONE ELSE.    IF YOU WILL LIVE LIKE NO ONE ELSE, LATER YOU CAN LIVE LIKE NO ONE ELSE.    IF YOU WILL LIVE LIKE NO ONE ELSE, LATER YOU CAN LIVE l\nAN\nHide It\nWhen you get the $1,000, hide it. You can’t keep the money handy because \nit will 

In [56]:
print(vectorstore.similarity_search("where to invest to maximise returns?", k=5)[0].page_content)

returns offered by investing well. Ignorance is not lack of intelligence; it is 
simply “not knowing.” Sadly, many intelligent but ignorant people seem 
to think that making a 12 percent rate of return on their money in a long-
term investment is impossible. And that if I state that there is a 12 percent 
rate of return available, then I have lied to them or misled them.
I recommend good growth-stock mutual funds in this book as a long-
term investment and dare to state that you should make 12 percent on 
your money over time. The supporting data for that bold statement can be 
found by looking at the historical averages of the Standard & Poor’s 500 
index. Widely regarded as the best single gauge of the US equities market, 
the S&P 500 is an index with five hundred of the largest companies in 
leading industries of the US economy. The S&P 500 has averaged 11.69 
percent per year for the last eighty-plus years, as of this writing. This


In [82]:
retriever = vectorstore.as_retriever(search_kwargs={"k":10}) #this will be on top of the vecor database

In [83]:
retriever.invoke("where to invest to maximise returns?")

[Document(id='a407c897-bfbe-43f7-9b26-92655ed9c505', metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2013-07-01T15:30:52-05:00', 'source': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'file_path': 'c:\\Users\\KAUSTHAB DUTTA\\Documents\\llmops_course\\project\\notebook\\data\\sample.pdf', 'total_pages': 266, 'format': 'PDF 1.6', 'title': 'The Total Money Makeover', 'author': 'Dave Ramsey', 'subject': '', 'keywords': '', 'moddate': '2013-07-15T14:28:01+05:30', 'trapped': '', 'modDate': "D:20130715142801+05'30'", 'creationDate': "D:20130701153052-05'00'", 'page': 15}, page_content='returns offered by investing well. Ignorance is not lack of intelligence; it is \nsimply “not knowing.” Sadly, many intelligent but ignorant people seem \nto think that making a 12 percent rate of return on their money in a long-\nterm investment is impossible. And that if I state that there is a 12 pe

In [65]:
prompt_template = """
        Answer the question based on the context provided below.
        If the context does not contain sufficient information, respond with:
        "I do not havr enough information about this"

        Context: {context}

        Question: {question}

        Answer:
"""

In [63]:
from langchain.prompts import PromptTemplate

In [67]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context","question"]
)

In [68]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n        Answer the question based on the context provided below.\n        If the context does not contain sufficient information, respond with:\n        "I do not havr enough information about this"\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:\n')

## Creating chain

In [69]:
from langchain_core.output_parsers import StrOutputParser

In [70]:
parser = StrOutputParser()

In [ ]:
def format_docs(docs): #tis function captures the page content
    return "\n\n".join([doc.page_content for doc in docs])

In [75]:
from langchain_core.runnables import RunnablePassthrough

In [76]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | StrOutputParser()
)

In [80]:
print(rag_chain.invoke("what is the fundamental rule of investing the book talks about"))

<think>
Okay, let's see. The user is asking about the fundamental rule of investing as discussed in the provided context. First, I need to go through the context carefully to extract relevant information.

The context starts by mentioning the author's investment approach, focusing on mutual funds with a good track record over five or ten years. They emphasize long-term thinking, not looking at short-term results. Then there's a part about recommending good growth-stock mutual funds and aiming for a 12% return, citing the S&P 500's historical average of 11.69% over 80 years. The author also talks about keeping investments simple unless you have over $10 million, using mutual funds and debt-free real estate. There's a mention of managing your own money and having a team that's smarter but you make the decisions.

Looking for the fundamental rule, the key points seem to be about long-term investment in growth funds, aiming for a 12% return based on historical averages, simplicity in inves

In [84]:
print(rag_chain.invoke("which stock has beaten the S&P 500"))

<think>
Okay, let's see. The user is asking which stock has beaten the S&P 500 based on the provided context. I need to check the context thoroughly.

Looking at the context, the author mentions several mutual funds they've invested in. For example, a Growth and Income Stock Mutual Fund that averaged 12.03% per year since 1934. Another one bought recently has averaged over 13.9% since 1973. There are others with 12.01%, 12.39%, and 11.72% since different years. The S&P 500 is mentioned as having an average of 11.69% over eighty years. 

The key here is that the mutual funds mentioned have higher average returns than the S&P 500. However, the question specifically asks about a stock, not a mutual fund. The context doesn't list any individual stocks, only mutual funds. The author talks about mutual funds outperforming the S&P 500, but not individual stocks. 

Since the user is asking about a stock, and the context doesn't mention any specific stocks by name, just mutual funds, the answer